In [ ]:
!vllm serve meta-llama/Llama-3.1-8B --max-model-len 8192

In [61]:
from datasets import load_dataset
import evaluate
import requests
import json
from tqdm.notebook import tqdm  # Use tqdm.notebook for Jupyter Notebook
import time

# Load test data
dataset = load_dataset("gigaword", split="test[:100]")  # Limit to 100 for fast eval

# Initialize ROUGE metric
rouge = evaluate.load('rouge')

In [ ]:
def summarize_with_vllm(document, max_tokens=50):
    prompt_template = (
        "You are an AI assistant specialized in summarizing news articles. "
        "Summarize the following news sentence into a concise headline.\n\n"

        "Here is an example:\n"
        "News: Japan 's nec corp. and UNK computer corp. of the united states said wednesday they had agreed to join forces in supercomputer sales.\n"
        "Headline: Nec UNK in computer sales tie-up\n\n"

        "Now summarize the following news:\n\n"

        "News: {document}\n\n"
        "Headline:"
    )
    
    prompt = prompt_template.format(document=document)

    payload = {
        "model": "meta-llama/Llama-3.1-8B",
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": 0.3,
        "stream": False
    }

    response = requests.post("http://localhost:8000/v1/completions", json=payload)

    if response.status_code == 200:
        result = response.json()
        summary = result['choices'][0]['text'].strip()
        return summary
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None


In [ ]:
# Generate summaries and evaluate
references = []
predictions = []

start = time.time()

# tqdm around dataset loop with a description and progress bar
for item in tqdm(dataset, desc="Summarizing", unit="example"):

    doc = item['document']
    ref_summary = item['summary']

    pred_summary = summarize_with_vllm(doc)

    if pred_summary:
        references.append(ref_summary)
        predictions.append(pred_summary)
    

end = time.time()

# Evaluate with ROUGE
results = rouge.compute(predictions=predictions, references=references)

print("vLLM (Llama-3.1-8B) Summarization Results:")

print(f"\nNumber of examples: {len(references)}")
print(f"\nElapsed time: {end - start:.2f} s")

print("\nROUGE Results:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")

Summarizing:   0%|          | 0/100 [00:00<?, ?example/s]

vLLM (Llama-3.1-8B) Summarization Results:

Number of examples: 98

Elapsed time: 271.17 s

ROUGE Results:
rouge1: 0.1682
rouge2: 0.0582
rougeL: 0.1498
rougeLsum: 0.1538
